In [1]:
import spacy
from spacy.matcher import Matcher

In [2]:
# Load the small english model
nlp = spacy.load("en_core_web_sm")

In [3]:
matcher = Matcher(nlp.vocab)
# If it looks like an email
pattern = [{"LIKE_EMAIL":True}]
# Givieng name
matcher.add("EMAIL_ADDRESS",[pattern])

In [4]:
doc = nlp("This is an email address: wmattingly@aol.com")
matches = matcher(doc)

In [5]:
print(matches)

[(16571425990740197027, 6, 7)]


In [9]:
print(nlp.vocab[matches[0][0]].text)

EMAIL_ADDRESS


In [23]:
with open ("wiki_mlk.txt", "r") as f:
    text = f.read()

In [24]:
print(text)

Martin Luther King Jr. (born Michael King Jr.; January 15, 1929 â€“ April 4, 1968) was an American Baptist minister and activist who became the most visible spokesman and leader in the American civil rights movement from 1955 until his assassination in 1968. King advanced civil rights through nonviolence and civil disobedience, inspired by his Christian beliefs and the nonviolent activism of Mahatma Gandhi. He was the son of early civil rights activist and minister Martin Luther King Sr.

King participated in and led marches for blacks' right to vote, desegregation, labor rights, and other basic civil rights.[1] King led the 1955 Montgomery bus boycott and later became the first president of the Southern Christian Leadership Conference (SCLC). As president of the SCLC, he led the unsuccessful Albany Movement in Albany, Georgia, and helped organize some of the nonviolent 1963 protests in Birmingham, Alabama. King helped organize the 1963 March on Washington, where he delivered his famou

In [25]:
# Load the english small model
nlp = spacy.load("en_core_web_sm")

In [26]:
# add porper noun to vocab
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN"}]
matcher.add("PROPER_NOUNS", [pattern])
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

103
(3232560085755078826, 0, 1) Martin
(3232560085755078826, 1, 2) Luther
(3232560085755078826, 2, 3) King
(3232560085755078826, 3, 4) Jr.
(3232560085755078826, 6, 7) Michael
(3232560085755078826, 7, 8) King
(3232560085755078826, 8, 9) Jr.
(3232560085755078826, 10, 11) January
(3232560085755078826, 16, 17) April
(3232560085755078826, 23, 24) American


#### Improving with multi word token

In [27]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}] # here op is Operator to determine how often to match a token pattern
matcher.add("PROPER_NOUNS", [pattern])
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

173
(3232560085755078826, 0, 1) Martin
(3232560085755078826, 0, 2) Martin Luther
(3232560085755078826, 1, 2) Luther
(3232560085755078826, 0, 3) Martin Luther King
(3232560085755078826, 1, 3) Luther King
(3232560085755078826, 2, 3) King
(3232560085755078826, 0, 4) Martin Luther King Jr.
(3232560085755078826, 1, 4) Luther King Jr.
(3232560085755078826, 2, 4) King Jr.
(3232560085755078826, 3, 4) Jr.


#### Greedy Longest Argument

In [28]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

62
(3232560085755078826, 470, 475) Martin Luther King Jr. Day
(3232560085755078826, 537, 542) Martin Luther King Jr. Memorial
(3232560085755078826, 0, 4) Martin Luther King Jr.
(3232560085755078826, 84, 88) Martin Luther King Sr
(3232560085755078826, 129, 133) Southern Christian Leadership Conference
(3232560085755078826, 248, 252) Director J. Edgar Hoover
(3232560085755078826, 6, 9) Michael King Jr.
(3232560085755078826, 326, 329) Nobel Peace Prize
(3232560085755078826, 423, 426) James Earl Ray
(3232560085755078826, 464, 467) Congressional Gold Medal


#### Sorting 

In [30]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}] 
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
# Sort with start of token
matches.sort(key = lambda x: x[1])
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

62
(3232560085755078826, 0, 4) Martin Luther King Jr.
(3232560085755078826, 6, 9) Michael King Jr.
(3232560085755078826, 10, 11) January
(3232560085755078826, 16, 17) April
(3232560085755078826, 23, 25) American Baptist
(3232560085755078826, 50, 51) King
(3232560085755078826, 70, 72) Mahatma Gandhi
(3232560085755078826, 84, 88) Martin Luther King Sr
(3232560085755078826, 90, 91) King
(3232560085755078826, 114, 115) King


#### Adding sequence

In [31]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"},{"POS":"VERB"}] # the next thing to be occured is a verb
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
# Sort with start of token
matches.sort(key = lambda x: x[1])
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

6
(3232560085755078826, 50, 52) King advanced
(3232560085755078826, 90, 92) King participated
(3232560085755078826, 114, 116) King led
(3232560085755078826, 248, 253) Director J. Edgar Hoover considered
(3232560085755078826, 323, 325) King won
(3232560085755078826, 486, 489) United States beginning


### Finding Quotes and Speaker

In [34]:
import json
with open("alice.json", "r") as f:
    data = json.load(f)

In [35]:
text = data[0][2][0]
print (text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'


In [36]:
text = text.replace("`", "'")
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [37]:
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH": "'", "OP": "+"},# start with quotation mark
           {"IS_ALPHA":True, "OP": "+"}, # has a series of alpha characters
           {"IS_PUNCT":True, "OP": "*"}, # has punctuation
           {"ORTH": "'"} # Closing quotation mark
          ] 
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
# Sort with start of token
matches.sort(key = lambda x: x[1])
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

2
(3232560085755078826, 47, 58) 'and what is the use of a book,'
(3232560085755078826, 60, 67) 'without pictures or conversation?'


In [42]:
speak_lemmas = ["think", "say"]
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH": "'", "OP": "+"},# start with quotation mark
           {"IS_ALPHA":True, "OP": "+"}, # has a series of alpha characters
           {"IS_PUNCT":True, "OP": "*"}, # has punctuation
           {"ORTH": "'"}, # Closing quotation mark
           {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}},
           {"POS": "PROPN", "OP":"+"}
          ] 
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
# Sort with start of token
matches.sort(key = lambda x: x[1])
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

1
(3232560085755078826, 47, 60) 'and what is the use of a book,' thought Alice
